In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("anthropic:claude-opus-4-1-20250805", temperature=0)
response = llm.invoke("Hola, como estas?")
response.text

'¡Hola! Estoy muy bien, gracias por preguntar. ¿Cómo estás tú? ¿En qué puedo ayudarte hoy?'

In [4]:
response = llm.invoke("Que clima hace en la ciudad de Bogota?")
response.text

'Bogotá tiene un **clima frío de montaña** con características muy particulares:\n\n## Características principales:\n\n🌡️ **Temperatura**\n- Promedio anual: 14°C (57°F)\n- Máximas: raramente superan los 20°C\n- Mínimas: pueden bajar hasta 5-7°C en madrugadas\n\n🌧️ **Lluvias**\n- Dos temporadas lluviosas: abril-mayo y octubre-noviembre\n- Dos temporadas secas: diciembre-marzo y julio-agosto\n- Precipitación anual: aproximadamente 800-1000 mm\n\n☁️ **Condiciones generales**\n- Ubicada a 2,640 metros sobre el nivel del mar\n- Cielo frecuentemente nublado\n- Humedad relativa alta (70-80%)\n- No tiene estaciones marcadas (primavera, verano, otoño, invierno)\n\n## Recomendaciones:\n- Llevar siempre una chaqueta o abrigo ligero\n- Tener paraguas o impermeable a mano\n- Vestirse en capas (el clima puede cambiar durante el día)\n- En las mañanas y noches hace más frío\n\nLos bogotanos suelen decir que pueden experimentar "todas las estaciones en un solo día" debido a la variabilidad del clima.'

In [6]:
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar los productos que necesitan.

Y tus productos son:
- Computadora
- Mouse
- Teclado
- Audifonos
- Mousepad
"""
messages = [
    ("system", system_prompt),
    ("user", "Dime los productos que ofreces en la tienda")
]
response = llm.invoke(messages)
response.text

'¡Hola! Bienvenido a nuestra tienda. Me da mucho gusto atenderte.\n\nContamos con los siguientes productos para ti:\n\n🖥️ **Computadora** - Equipos completos para todas tus necesidades de trabajo y entretenimiento\n\n🖱️ **Mouse** - Para un control preciso y cómodo de tu computadora\n\n⌨️ **Teclado** - Teclados de calidad para una escritura eficiente\n\n🎧 **Audífonos** - Para disfrutar de tu música, videollamadas o gaming con el mejor sonido\n\n🗒️ **Mousepad** - Superficies ideales para mejorar la precisión y comodidad al usar tu mouse\n\n¿Hay algún producto en particular que te interese? ¡Con gusto te puedo dar más información y ayudarte a encontrar exactamente lo que necesitas!'

In [ ]:
from langchain_core.tools import tool
import requests

@tool("get_products", description="Get the products that the store offers filter by price")
def get_products():
    # Connnect with API
    """Get the products that the store offers"""
    response = requests.get("https://api.escuelajs.co/api/v1/products")
    products = response.json()
    return "".join([f"{product['title']} - {product['price']}" for product in products])



In [33]:
get_products.invoke({})

"Majestic Mountain Graphic T-Shirt - 44Classic Red Pullover Hoodie - 10Classic Black Hooded Sweatshirt - 79Classic Comfort Fit Joggers - 25Classic Comfort Drawstring Joggers - 79Classic Red Jogger Sweatpants - 98Classic Navy Blue Baseball Cap - 61Classic Blue Baseball Cap - 86Classic Red Baseball Cap - 35Classic Black Baseball Cap - 58Classic Olive Chino Shorts - 84Classic High-Waisted Athletic Shorts - 43Classic White Crew Neck T-Shirt - 39Classic White Tee - Timeless Style and Comfort - 73Classic Black T-Shirt - 35Sleek White & Orange Wireless Gaming Controller - 69Sleek Wireless Headphone & Inked Earbud Set - 44Sleek Comfort-Fit Over-Ear Headphones - 28Efficient 2-Slice Toaster - 48Sleek Wireless Computer Mouse - 10Sleek Modern Laptop with Ambient Lighting - 43Sleek Modern Laptop for Professionals - 97Stylish Red & Silver Over-Ear Headphones - 39Sleek Mirror Finish Phone Case - 27Sleek Smartwatch with Vibrant Display - 16Sleek Modern Leather Sofa - 53Mid-Century Modern Wooden Dining

In [35]:
@tool("get_weather", description="Get the weather of a city")
def get_weather(city: str):
    response = requests.get(f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1")
    data = response.json()
    latitude = data["results"][0]["latitude"]
    longitude = data["results"][0]["longitude"]
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true")
    data = response.json()
    response = f"The weather in {city} is {data["current_weather"]["temperature"]}C with {data["current_weather"]["windspeed"]}km/h of wind."
    return response

get_weather.invoke({"city": "bogota"})

'The weather in bogota is 18.5C with 5.8km/h of wind.'

In [38]:
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar los productos que necesitan y dar el clima de la ciudad

Tus tools son:
- get_products: para obtener los productos que ofreces en la tienda
- get_weather: para obtener el clima de la ciudad
"""
messages = [
    ("system", system_prompt),
    ("user", "Dime los productos que ofreces en la tienda")
]
llm_with_tools = llm.bind_tools([get_products, get_weather])
response = llm_with_tools.invoke(messages)
response.tool_calls

[{'name': 'get_products',
  'args': {},
  'id': 'toolu_018f6L8Q7uDAptsZKu2Vmavs',
  'type': 'tool_call'}]

In [39]:
messages = [
    ("system", system_prompt),
    ("user", "Hola, que tal?")
]
response = llm_with_tools.invoke(messages)
response.text

'¡Hola! ¡Muy bien, gracias por preguntar! 👋\n\nSoy tu asistente de ventas y estoy aquí para ayudarte. Puedo:\n\n1. **Mostrarte los productos** que tenemos disponibles en nuestra tienda\n2. **Consultar el clima** de cualquier ciudad que necesites\n\n¿En qué puedo ayudarte hoy? ¿Te gustaría ver nuestros productos o necesitas información del clima de alguna ciudad?'

In [42]:
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar los productos que necesitan y dar el clima de la ciudad

Tus tools son:
- get_products: para obtener los productos que ofreces en la tienda
- get_weather: para obtener el clima de la ciudad
"""
messages = [
    ("system", system_prompt),
    ("user", "Cual es el clima en la capital de Bolivia?")
]
llm_with_tools = llm.bind_tools([get_products, get_weather])
response = llm_with_tools.invoke(messages)
response.tool_calls

[{'name': 'get_weather',
  'args': {'city': 'La Paz'},
  'id': 'toolu_01UesvbowYfK9eG8X4H8RLYp',
  'type': 'tool_call'}]